---
Imports

In [45]:
import numpy as np

---
Código de la Capa

In [46]:
class Capa:
    def __init__(self, num_neuronas, num_entradas, fun_act):
        self.peso_input = np.random.random((num_entradas+1, num_neuronas))
        self.fun_act = fun_act

    # Entra una matriz de 1 x n y sale una matriz de 1 x n
    def get_output(self, input):
        input = np.insert(input, 0, 1)
        if (input.shape[0] == self.peso_input.shape[0]):
            self.suma = np.matmul(input, self.peso_input)
            return self.step_function(self.suma)
        else: return "Ha habido un error: No coinciden el número de neuronas"

    # Decide qué hacer según la función de activación
    def step_function(self, input):
        if self.fun_act == 1:
            return self.sf_threshold(input)
        elif self.fun_act == 2:
            return self.sf_sigmoid(input)
        elif self.fun_act == 3:
            return self.sf_hyperbolic_tangent(input)
        elif self.fun_act == 4:
            return self.sf_relu(input)
        else:
            return "Ha habido un error: Función de activación incorrecta."

    def sf_threshold(self, input):
        output = input.copy()
        output[output<=0] = 0
        output[output>0] = 1
        return output

    def sf_sigmoid(self, input):
        return 1/(1+np.exp(-input))

    def sf_hyperbolic_tangent(self, input):
        return (np.exp(input) - np.exp(-input)) / (np.exp(input) + np.exp(-input))

    def sf_relu(self, input):
        return np.maximum([0], input)

    # Función para mostrar los datos de la capa
    def print(self):
        print("Inputs: \n", self.peso_input)
        print("Input shape: \n", self.peso_input.shape)
        print("F Act: \n", self.fun_act)

---

In [47]:
class Red_Neuronal():
    def __init__(self,num_neuronas_capa, f_act_capa, num_inputs_1):
        self.lista_capas = []
        c = Capa(num_neuronas_capa[0], num_inputs_1, f_act_capa[0])
        self.lista_capas.append(c)
        for i in range(1, len(num_neuronas_capa)):
            c = Capa(num_neuronas_capa[i], num_neuronas_capa[i-1], f_act_capa[i])
            self.lista_capas.append(c)

    # Función para hacer los cálculos
    def calc(self, inputs):
        for c in self.lista_capas:
            inputs = c.get_output(inputs)
        return inputs

    # Función para entrenar
    def train(self, input, out_esp , ratio):
        inputs, sumas = [], []
        for c in self.lista_capas:
            input = c.get_output(input)
            sumas.append(c.suma)
            inputs.append(input)

        # Derivada según la función de activación
        df_act = self.f_act(sumas)

        delta = self.d_ecm(self.f_ecm(np.array(input), out_esp)) * df_act # derivada f_coste * derivada f_activación

        pesos = self.lista_capas[len(self.lista_capas)-1].peso_input[1:len(self.lista_capas[len(self.lista_capas)-1].peso_input)] # pesos

        print("pesos:")
        print(pesos)
        pesos = pesos - delta * pesos * ratio
        print(pesos)

        bias = self.lista_capas[0].peso_input[0] # bias
        
        print("bias")
        print(bias)
        bias = bias - delta * bias * ratio
        print(bias)


    # Derivada según la función de activación
    def f_act(self, sumas):
        if (self.lista_capas[len(self.lista_capas)-1].fun_act) == 2:
            return self.d_sigmoid(sumas[len(sumas)-1])
        if (self.lista_capas[len(self.lista_capas)-1].fun_act) == 3:
            return self.d_hyperbolic_tangent(sumas[len(sumas)-1])
        if (self.lista_capas[len(self.lista_capas)-1].fun_act) == 4:
            return self.d_relu(sumas[len(sumas)-1])

    def f_ecm(self, r_esp, r_obt):
        ecm = pow((r_esp - r_obt).sum(), 2)/len(r_esp)
        return ecm

    def d_ecm(self, input):
        return np.sqrt(input)

    def d_sigmoid(self, input):
        return input * (1 - input)

    def d_hyperbolic_tangent(self, input):
        return input * (1 - input) # input

    def d_relu(self, input):
        # if input <= 0: return 0
        # else: return 1
        output = input.copy()
        output[output<=0] = 0
        output[output>0] = 1
        return output

    # Función para mostrar los datos de la red neuronal
    def print(self):
        c = 1
        for capa in self.lista_capas:
            print("Capa c: ")
            capa.print()
            print("---------")
            c += 1

---
Pruebas

In [48]:
rn1 = Red_Neuronal([3,2,3], [4,2,3], 4)
rn1.train([[4, -2, 3, -5]], [[0.7, 0.9, 0.5]], 0.75)

pesos:
[[0.37792522 0.17601741 0.64107304]
 [0.21883049 0.73790412 0.14859818]]
[[0.42751591 0.20298751 0.64386474]
 [0.24754505 0.85096877 0.14924529]]
bias
[0.08255422 0.45292343 0.52430374]
[0.09338684 0.52232218 0.52658693]


---

Error Cuadrático Medio: $$ECM = 1/N * Σ_i^N(RE_i-RO_i)^2$$
derivada del Error Cuadrático Medio: $$dECM = 1/N * Σ_i^N(RE_i-RO_i)$$
derivada función Sigmoide: $$x * (1-x)$$
derivada función Hiperbólica: $$dtanhip = 1 - (tanhip(x)^2)$$
derivada función ReLu: $$f(x) = { 0, x <= 0; 1, x > 0}$$